In [5]:
import glob
glob.glob('/home/lenovo/Documents/image processing studies/tsv/*.tsv')


['/home/lenovo/Documents/image processing studies/tsv/invoicee.tsv']

In [6]:
from PIL import Image      
import os.path, sys
import matplotlib.pyplot as plt

In [10]:
#read tsv file
# importing pandas module 
import pandas as pd 
  
# making data frame from csv file 
data = pd.read_csv('/home/lenovo/Documents/image processing studies/tsv/invoicee.tsv', sep='\t', encoding = 'unicode_escape') 
  
# dropping passed columns 
# data.drop(["Unnamed: 2"], axis = 1, inplace = True) 
  
# display 
data 


,Unnamed: 0,Unnamed: 1
0,am,O
1,aZO,O
2,ni,O
3,N,O
4,Tax,O
5,Invoice/Bill,O
6,of,O
7,Supply/Cash,O
8,Memo,O
9,(Original,O


In [11]:
#save as csv 
data.to_csv(r'ner_dataset1.csv', index = False)

In [12]:
#csv to tsv
import csv

with open('ner_dataset1.csv','r') as csvin, open('ner_dataset1.txt', 'w') as tsvout:
     csvin = csv.reader(csvin)
     tsvout = csv.writer(tsvout, delimiter='\t')

     for row in csvin:
          tsvout.writerow(row)

In [17]:
# Convert .tsv file to dataturks json format.
import json
import logging
import sys
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='\t':
                
                word,entity=line.split('\t')
                s+=word+" "
                #  print(s)
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d)
                start+=len(word)+1
            else:
                
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])):
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                            
                             
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)                          
                print(data_dict)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None
tsv_to_json_format(r"ner_dataset1.txt",r'ner_dataset1.json','abc')

{'content': 'Unnamed: 0 am aZO ni N Tax Invoice/Bill of Supply/Cash Memo (Original for Recipient) Signature Not Verified ff Digitally signed by DS CLOUDTAIL iA PRIVATE LIMITED 1 Date: 2018.09.23 18:09:50 UTC Reason: Invoice Sold By : Billing Address : Cloudtail India Private Limited Krishna Prasad â\x80\x9d Sy No. 524/1,2,3,4,6, 525/1,2,3,4,5,6, Teknowmics knowledge solution, 8F Noel Focus, 526/3,4,5,6,527 of madivala village and Sy no.51/1 chittetukara,, Kakkanad of thatanahalli village, kasaba hobli, anekal taluk,, Kakkanad, KERALA, 682037 Bangalore urban district IN Bangalore, Karnataka, 562107 IN Shipping Address : Krishna Prasad PAN No: AAQCS42590 Rathnam GST Registration No: 29AAQCS4259Q1Z6 Gokulam, Chemmanamkad, kanjikode, palakkad KANJIKODE INDUSTRIAL AREA, KERALA, 678621 IN Order Number: 171-0593478-3651547 Invoice Number : BLR7-2957015 Order Date: 23.09.2018 Invoice Details : KA-BLR7-1004-1819 Invoice Date : 23.09.2018 a 2 ee Rate|Type|Amount/Amount 1 |Prestige Deluxe Plus Ha

In [18]:

#spacy format
import plac
import logging
import argparse
import sys
import os
import json
import pickle

# @plac.annotations(input_file=("/content/demo.json", "option", "i", str), output_file=("/content/out.json", "option", "o", str))
def main(input_file='ner_dataset1.json', output_file='out.json'):
    try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        # print(training_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None
main()

In [23]:
from __future__ import unicode_literals, print_function
import pickle
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
import os
from os import listdir



# New entity labels
# Specify the new entity labels which you want to add here
LABEL = [u'B-invoiceno', u'B-invoicedate', u'B-customer', u'I-customer', u'B-vendor', u'I-vendor', u'B-product', u'I-product', u'B-address', u'I-address', u'B-amount']

with open ('out.json', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)

# @plac.annotations(
#     model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
#     new_model_name=("New model name for model meta.", "option", "nm", str),
#     output_dir=("Optional output directory", "option", "o", Path),
#     n_iter=("Number of training iterations", "option", "n", int))

def main(model=None, new_model_name='new_model', output_dir='/home/lenovo/Documents/teknowmics/', n_iter=10):
    """Setting up the pipeline and entity recognizer, and training the new entity."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spacy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    else:
        ner = nlp.get_pipe('ner')
    # for i in LABEL:
    #     ner.add_label(i)   # Add new entity labels to entity recognizer

    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.entity.create_optimizer()

    # Get names of other pipes to disable them during training to train only NER
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
                           losses=losses)
            print('Losses', losses)
        # save model to output directory
    output_dir1="/home/lenovo/Documents/teknowmics/model"
    if output_dir1 is not None:
        output_dir1 = Path(output_dir1)
        if not output_dir1.exists():
            output_dir1.mkdir()
        nlp.to_disk(output_dir1)
        print("Saved model to", output_dir1)
        
main()

Created blank 'en' model
Losses {'ner': 6798.677579432726}
Losses {'ner': 2012.5855028274527}
Losses {'ner': 1794.825552713126}
Losses {'ner': 1666.3802489736117}
Losses {'ner': 1599.6168642917182}
Losses {'ner': 1526.3172220255365}
Losses {'ner': 1620.9591462707613}
Losses {'ner': 1359.1019457193906}
Losses {'ner': 1305.4107606959078}
Losses {'ner': 1235.020169561115}
Saved model to /home/lenovo/Documents/ner advanved/model
